In [18]:
import json
import collections

In [19]:
bounding_local_parsed_path = 'bounding_local_parsed.json'
bounding_university_parsed_path = 'bounding_university_parsed.json'
bounding_college_parsed_path = 'bounding_college_parsed.json'

In [20]:
with open(bounding_local_parsed_path, 'r') as read_file:
    bounding_local = json.load(read_file)
with open(bounding_university_parsed_path, 'r') as read_file:
    bounding_university = json.load(read_file)
with open(bounding_college_parsed_path, 'r') as read_file:
    bounding_college = json.load(read_file)

In [21]:
def arbitraryflatten(iterable):
    iterator, sentinel, stack = iter(iterable), object(), []
    while True:
        value = next(iterator, sentinel)
        if value is sentinel:
            if not stack:
                break
            iterator = stack.pop()
        elif isinstance(value, str):
            yield value
        else:
            try:
                new_iterator = iter(value)
            except TypeError:
                yield value
            else:
                stack.append(iterator)
                iterator = new_iterator

In [22]:
def checkshared(alist):
    import copy
    namesdict = {}
    passtest = True
    for bound in alist:
        nodelist = bound['nodelist']
        thename = bound['name']
        if thename not in namesdict:
            namesdict[thename] = copy.deepcopy(bound)
        else:
            #compare how far apart they are
            fromnamesdict = copy.deepcopy(namesdict[thename]['nodelist'])
            frombound = copy.deepcopy(bound['nodelist'])
            fromnamesdictflattened = list(arbitraryflatten(fromnamesdict))
            fromnamesdictflattenedlat = [element for i, element in enumerate(fromnamesdictflattened) if i%2==0]
            fromnamesdictflattenedlon = [element for i, element in enumerate(fromnamesdictflattened) if i%2!=0]
            fromnamesdictzipped = list(zip(fromnamesdictflattenedlat, fromnamesdictflattenedlon))
            
            fromboundflattened = list(arbitraryflatten(frombound))
            fromboundflattenedlat = [element for i, element in enumerate(fromboundflattened) if i%2==0]
            fromboundflattenedlon = [element for i, element in enumerate(fromboundflattened) if i%2!=0]
            fromboundzipped = list(zip(fromboundflattenedlat, fromboundflattenedlon))
            
            aset = set()
            for el in fromboundzipped:
                aset.add(el)
            shared = aset.intersection(fromnamesdictzipped)
            
            if shared:
                passtest = False
#                 print(namesdict[thename])
#                 print("")
#                 print(bound)
#                 print("")
#                 print(shared)
#                 print("")
    return passtest

In [23]:
def combineshared(alist):
    import copy
    namesdict = {}
    finallist = []
    copiedalist = copy.deepcopy(alist)
    for bound in copiedalist:
        nodelist = bound['nodelist']
        thename = bound['name']
        if thename not in namesdict:
            namesdict[thename] = copy.deepcopy(bound)
            finallist.append(namesdict[thename])
        else:
            #compare how far apart they are
            fromnamesdict = copy.deepcopy(namesdict[thename]['nodelist'])
            frombound = copy.deepcopy(bound['nodelist'])
            fromnamesdictflattened = list(arbitraryflatten(fromnamesdict))
            fromnamesdictflattenedlat = [element for i, element in enumerate(fromnamesdictflattened) if i%2==0]
            fromnamesdictflattenedlon = [element for i, element in enumerate(fromnamesdictflattened) if i%2!=0]
            fromnamesdictzipped = list(zip(fromnamesdictflattenedlat, fromnamesdictflattenedlon))
            
            fromboundflattened = list(arbitraryflatten(frombound))
            fromboundflattenedlat = [element for i, element in enumerate(fromboundflattened) if i%2==0]
            fromboundflattenedlon = [element for i, element in enumerate(fromboundflattened) if i%2!=0]
            fromboundzipped = list(zip(fromboundflattenedlat, fromboundflattenedlon))
            
            aset = set()
            for el in fromboundzipped:
                aset.add(el)
            shared = aset.intersection(fromnamesdictzipped)
#             if bound['name'] == 'Hangar 2':
#                 print(shared)
#                 if shared:
#                     indexoffirstbound = finallist.index(namesdict[thename])
#                     finallist[indexoffirstbound]['nodelist'].extend(copy.deepcopy(bound['nodelist']))
#                     bound['type'] = 'relation'
#                     namesdict[thename]['type'] = 'relation'
#                     finallist[indexoffirstbound]['type'] = 'relation'
#                     print(finallist[indexoffirstbound]['nodelist'])
#                     print(copy.deepcopy(bound['nodelist']))
#                 else:
#                     finallist.append(copy.deepcopy(bound))
#                     print("appended")
#                     print(copy.deepcopy(bound))
            
            if shared:
                indexoffirstbound = finallist.index(namesdict[thename])
                if finallist[indexoffirstbound]['type'] == 'way':
                    finallist[indexoffirstbound]['nodelist'] = [finallist[indexoffirstbound]['nodelist']]
                if bound['type'] == 'way':
                    bound['nodelist'] = [bound['nodelist']]
                finallist[indexoffirstbound]['nodelist'].extend(copy.deepcopy(bound['nodelist']))
                bound['type'] = 'relation'
                namesdict[thename]['type'] = 'relation'
                finallist[indexoffirstbound]['type'] = 'relation'
                        
            else:
                finallist.append(copy.deepcopy(bound))
                #pop the other one
    return finallist

In [24]:
#redo until passes
def surelycombinedshared(alist):
    redocount = 0
    while not checkshared(alist):
        alist = combineshared(alist)
        redocount += 1
    print(redocount)
    return alist

In [25]:
try_bounding_college = surelycombinedshared(bounding_college)
try_bounding_university = surelycombinedshared(bounding_university)
try_bounding_local = surelycombinedshared(bounding_local)

2
4
0


In [ ]:
def forwardreversecomparison(thenodelist):
    newnodelistfoward = copy.deepcopy(thenodelist)
    safewaylistsfoward = []

    firstmembersfoward = []
    lastmembersfoward = []

    for index, waylist in enumerate(newnodelistfoward):
        for wayindex, element in enumerate(waylist):
            waylist[wayindex] = tuple(element)

    for index, waylist in enumerate(newnodelistfoward):
        if waylist[-1] == waylist[0]:
            safewaylistsfoward.append(copy.deepcopy(waylist))
            newnodelistfoward.pop(index)
        else:
            firstmembersfoward.append(copy.deepcopy(waylist[0]))
            lastmembersfoward.append(copy.deepcopy(waylist[-1]))

    nextfirstmembersfoward = []
    nextlastmembersfoward = []

    fowardcount = 0
    while nextfirstmembersfoward != firstmembersfoward and nextlastmembersfoward != lastmembersfoward:
        nextfirstmembersfoward = []
        nextlastmembersfoward = []
        for index, waylist in enumerate(newnodelistfoward):         
            if waylist[-1] == waylist[0]:
                safewaylistsfoward.append(copy.deepcopy(waylist))
                newnodelistfoward.pop(index)
            else:
                nextfirstmembersfoward.append(copy.deepcopy(waylist[0]))
                nextlastmembersfoward.append(copy.deepcopy(waylist[-1]))

        firstmembersfoward = copy.deepcopy(nextfirstmembersfoward)
        lastmembersfoward = copy.deepcopy(nextlastmembersfoward)

        asetfoward = set()
        for el in firstmembersfoward:
            asetfoward.add(el)
        sharedfoward = asetfoward.intersection(lastmembersfoward)

        while sharedfoward != set():
            fowardcount += 1
            sharefoward = sharedfoward.pop()
            index1foward = firstmembersfoward.index(sharefoward)
            index2foward = lastmembersfoward.index(sharefoward)

            newnodelistfoward[index1foward].pop(0)

            newnodelistfoward[index2foward].extend(copy.deepcopy(newnodelistfoward[index1foward]))

            newnodelistfoward.pop(index1foward)

            firstmembersfoward = []
            lastmembersfoward = []
            for index, waylist in enumerate(newnodelistfoward):         
                if waylist[-1] == waylist[0]:
                    safewaylistsfoward.append(copy.deepcopy(waylist))
                    newnodelistfoward.pop(index)
                else:
                    firstmembersfoward.append(copy.deepcopy(waylist[0]))
                    lastmembersfoward.append(copy.deepcopy(waylist[-1]))
            asetfoward = set()
            for el in firstmembersfoward:
                asetfoward.add(el)
            sharedfoward = asetfoward.intersection(lastmembersfoward)
    newnodelistreverse = copy.deepcopy(bound['nodelist'])
    safewaylistsreverse = []

    firstmembersreverse = []
    lastmembersreverse = []

    for index, waylist in enumerate(newnodelistreverse):
        for wayindex, element in enumerate(waylist):
            waylist[wayindex] = tuple(element)

    for index, waylist in enumerate(newnodelistreverse):
        if waylist[-1] == waylist[0]:
            safewaylistsreverse.append(copy.deepcopy(waylist))
            newnodelistreverse.pop(index)
        else:
            firstmembersreverse.append(copy.deepcopy(waylist[0]))
            lastmembersreverse.append(copy.deepcopy(waylist[-1]))

    nextfirstmembersreverse = []
    nextlastmembersreverse = []

    reversecount = 0
    while nextfirstmembersreverse != firstmembersreverse and nextlastmembersreverse != lastmembersreverse:
        nextfirstmembersreverse = []
        nextlastmembersreverse = []
        for index, waylist in enumerate(newnodelistreverse):         
            if waylist[-1] == waylist[0]:
                safewaylistsreverse.append(copy.deepcopy(waylist))
                newnodelistreverse.pop(index)
            else:
                nextfirstmembersreverse.append(copy.deepcopy(waylist[0]))
                nextlastmembersreverse.append(copy.deepcopy(waylist[-1]))

        firstmembersreverse = copy.deepcopy(nextfirstmembersreverse)
        lastmembersreverse = copy.deepcopy(nextlastmembersreverse)

        asetreverse = set()
        for el in firstmembersreverse:
            asetreverse.add(el)
        sharedreverse = asetreverse.intersection(lastmembersreverse)

        while sharedreverse != set():
            reversecount += 1
            sharereverse = sharedreverse.pop()
            index1reverse = firstmembersreverse.index(sharereverse)
            index2reverse = lastmembersreverse.index(sharereverse)

            newnodelistreverse[index1reverse].pop(0)

            newnodelistreverse[index2reverse].extend(copy.deepcopy(newnodelistreverse[index1reverse]))

            newnodelistreverse[index2reverse] = newnodelistreverse[index2reverse][::-1]

            newnodelistreverse.pop(index1reverse)

            firstmembersreverse = []
            lastmembersreverse = []
            for index, waylist in enumerate(newnodelistreverse):         
                if waylist[-1] == waylist[0]:
                    safewaylistsreverse.append(copy.deepcopy(waylist))
                    newnodelistreverse.pop(index)
                else:
                    firstmembersreverse.append(copy.deepcopy(waylist[0]))
                    lastmembersreverse.append(copy.deepcopy(waylist[-1]))
            asetreverse = set()
            for el in firstmembersreverse:
                asetreverse.add(el)
            sharedreverse = asetreverse.intersection(lastmembersreverse)

    if fowardcount >= reversecount:
        newnodelist = newnodelistfoward
        safewaylists = safewaylistsfoward
        bestcount = fowardcount
    else:
        newnodelist = newnodelistreverse
        safewaylists = safewaylistsreverse
        bestcount = reversecount
    return newnodelist, safewaylists, bestcount

In [13]:
def mergeways(alist):
    import copy
    finalnodelist = []
    alist_copy = copy.deepcopy(alist)
    for bound in alist_copy:
        if bound['type'] == 'relation':
            newnodelist, safewaylists, bestcount = forwardreversecomparison(bound['nodelist'])
            #######################
            firstmembers = []
            lastmembers = []
            for index, waylist in enumerate(newnodelist):
                if waylist[-1] == waylist[0]:
                    safewaylists.append(copy.deepcopy(waylist))
                    newnodelist.pop(index)
                else:
                    firstmembers.append(copy.deepcopy(waylist[0]))
                    lastmembers.append(copy.deepcopy(waylist[-1]))
                    
                    
            nextfirstmembers = []
            nextlastmembers = []
            #########################
            while nextfirstmembers != firstmembers and nextlastmembers != lastmembers:
               
                nextfirstmembers = []
                nextlastmembers = []
                for index, waylist in enumerate(newnodelist):         
                    if waylist[-1] == waylist[0]:
                        safewaylists.append(copy.deepcopy(waylist))
                        newnodelist.pop(index)
                    else:
                        nextfirstmembers.append(copy.deepcopy(waylist[0]))
                        nextlastmembers.append(copy.deepcopy(waylist[-1]))
                
                firstmembers = copy.deepcopy(nextfirstmembers)
                lastmembers = copy.deepcopy(nextlastmembers)
                
                ######################## firstmembers
                
                firstmemdups = [item for item, count in collections.Counter(firstmembers).items() if count > 1]
                while firstmemdups:
                    first = firstmemdups.pop()
                    
                    index1 = firstmembers.index(first)
                    index2 = [i for i, el in enumerate(firstmembers) if el == first][1]
                    
                    #$$$$$$$$$$$$ try both
                    templist1 = copy.deepcopy(newnodelist)
                    templist1[index1] = templist1[index1][::-1]
                    
                    templist1, templist1safe, templist1bestcount = forwardreversecomparison(templist1)
                    
                    templist2 = copy.deepcopy(newnodelist)
                    templist2[index2] = templist2[index2][::-1]
                    
                    templist2, templist2safe, templist2bestcount = forwardreversecomparison(templist2)
                    
                    if templist1bestcount >= templist2bestcount:
                        newnodelist = templist1
                        safewaylists.extend(templist1safe)
                    else:
                        newnodelist = templist2
                        safewaylists.extend(templist2safe)
                    
                    firstmembers = []
                    lastmembers = []
                    for index, waylist in enumerate(newnodelist):         
                        if waylist[-1] == waylist[0]:
                            safewaylists.append(copy.deepcopy(waylist))
                            newnodelist.pop(index)
                        else:
                            firstmembers.append(copy.deepcopy(waylist[0]))
                            lastmembers.append(copy.deepcopy(waylist[-1]))
                    firstmemdups = [item for item, count in collections.Counter(firstmembers).items() if count > 1]
                 
                
                #########
                firstmembers = []
                lastmembers = []
                for index, waylist in enumerate(newnodelist):         
                    if waylist[-1] == waylist[0]:
                        safewaylists.append(copy.deepcopy(waylist))
                        newnodelist.pop(index)
                    else:
                        firstmembers.append(copy.deepcopy(waylist[0]))
                        lastmembers.append(copy.deepcopy(waylist[-1]))
                ######################## lastmembers
                lastmemedups = [item for item, count in collections.Counter(lastmembers).items() if count > 1]
                while lastmemedups:
                    
                    last = lastmemedups.pop()
                    
                    index1 = lastmembers.index(last)
                    index2 = [i for i, el in enumerate(lastmembers) if el == last][1]


                    #$$$$$$$$$$$$ try both
                    templist1 = copy.deepcopy(newnodelist)
                    templist1[index1] = templist1[index1][::-1]
                    
                    templist1, templist1safe, templist1bestcount = forwardreversecomparison(templist1)
                    
                    templist2 = copy.deepcopy(newnodelist)
                    templist2[index2] = templist2[index2][::-1]
                    
                    templist2, templist2safe, templist2bestcount = forwardreversecomparison(templist2)
                    
                    if templist1bestcount >= templist2bestcount:
                        newnodelist = templist1
                        safewaylists.extend(templist1safe)
                    else:
                        newnodelist = templist2
                        safewaylists.extend(templist2safe)

                    firstmembers = []
                    lastmembers = []
                    for index, waylist in enumerate(newnodelist):         
                        if waylist[-1] == waylist[0]:
                            safewaylists.append(copy.deepcopy(waylist))
                            newnodelist.pop(index)
                        else:
                            firstmembers.append(copy.deepcopy(waylist[0]))
                            lastmembers.append(copy.deepcopy(waylist[-1]))
                    lastmemedups = [item for item, count in collections.Counter(lastmembers).items() if count > 1]    
            newnodelist.extend(safewaylists)
            finalnodelist.append(newnodelist)
        if bound['type'] == 'way':
            nodelist = bound['nodelist']
            newnodelist = copy.deepcopy(nodelist)
            finalnodelist.append(newnodelist)
    return finalnodelist

In [14]:
def applyandcopymergeways(alist):
    import copy
    alist_copy = copy.deepcopy(alist)
    allnodelists = mergeways(alist_copy)
    print(len(allnodelists))
    for nodelistindex, nodelist in enumerate(allnodelists):
        alist_copy[nodelistindex]['nodelist'] = nodelist
        if len(nodelist) > 1:
            alist_copy[nodelistindex]['type'] = 'relation'
            try:
                alist_copy[nodelistindex]['nodelist'][0][0][0]
            except TypeError as notfullynested:
                alist_copy[nodelistindex]['type'] = 'way'
        elif len(nodelist) == 1 and alist_copy[nodelistindex]['type'] == 'relation':
            alist_copy[nodelistindex]['nodelist'] = nodelist[0]
            alist_copy[nodelistindex]['type'] = 'way'
    return alist_copy

In [15]:
new_bounding_college = applyandcopymergeways(try_bounding_college)
new_bounding_university = applyandcopymergeways(try_bounding_university)
new_bounding_local = applyandcopymergeways(try_bounding_local)

NameError: name 'try_bounding_college' is not defined

In [44]:
def adjusttotuples(alist):
    import copy
    alist_copy = copy.deepcopy(alist)
    for bound in alist_copy:
        if bound['type'] == 'relation':
            for waylist in bound['nodelist']:
                for pairindex, pair in enumerate(waylist):
                    waylist[pairindex] = tuple(pair)
        if bound['type'] == 'way':
            for pairindex, pair in enumerate(bound['nodelist']):
                bound['nodelist'][pairindex] = tuple(pair)
    return alist_copy

In [45]:
tuple_bounding_college = adjusttotuples(new_bounding_college)
tuple_bounding_university = adjusttotuples(new_bounding_university)
tuple_bounding_local = adjusttotuples(new_bounding_local)

In [46]:
with open(bounding_local_parsed_path[:-5] + '_withdups.json', 'w') as write_file:
    json.dump(tuple_bounding_local, write_file)
with open(bounding_university_parsed_path[:-5] + '_withdups.json', 'w') as write_file:
    json.dump(tuple_bounding_university, write_file)
with open(bounding_college_parsed_path[:-5] + '_withdups.json', 'w') as write_file:
    json.dump(tuple_bounding_college, write_file)

# Start Here

In [1]:
import json
import collections

In [2]:
bounding_local_parsed_path = 'bounding_local_parsed_withdups.json'
bounding_university_parsed_path = 'bounding_university_parsed_withdups.json'
bounding_college_parsed_path = 'bounding_college_parsed_withdups.json'

In [3]:
with open(bounding_local_parsed_path, 'r') as read_file:
    bounding_local = json.load(read_file)
with open(bounding_university_parsed_path, 'r') as read_file:
    bounding_university = json.load(read_file)
with open(bounding_college_parsed_path, 'r') as read_file:
    bounding_college = json.load(read_file)

In [4]:
def arbitraryflatten(iterable):
    iterator, sentinel, stack = iter(iterable), object(), []
    while True:
        value = next(iterator, sentinel)
        if value is sentinel:
            if not stack:
                break
            iterator = stack.pop()
        elif isinstance(value, str):
            yield value
        else:
            try:
                new_iterator = iter(value)
            except TypeError:
                yield value
            else:
                stack.append(iterator)
                iterator = new_iterator

In [5]:
def adjusttotuples(alist):
    import copy
    alist_copy = copy.deepcopy(alist)
    for bound in alist_copy:
        if bound['type'] == 'relation':
            for waylist in bound['nodelist']:
                for pairindex, pair in enumerate(waylist):
                    waylist[pairindex] = tuple(pair)
        if bound['type'] == 'way':
            for pairindex, pair in enumerate(bound['nodelist']):
                bound['nodelist'][pairindex] = tuple(pair)
    return alist_copy

In [6]:
tuple_bounding_college = adjusttotuples(bounding_college)
tuple_bounding_university = adjusttotuples(bounding_university)
tuple_bounding_local = adjusttotuples(bounding_local)

In [9]:
def testnamestuff(alist):
    import copy
    alist_copy = copy.deepcopy(alist)
    badrelcount = 0
    badreldups = 0
    badwaycount = 0
    badwaydups = 0
    for probindex, bound in enumerate(alist_copy):
        newnodelist = bound['nodelist']
        if bound['type'] == 'relation':
            firstmembers = []
            lastmembers = []
            #print(bound)
            for i, waylist in enumerate(newnodelist):
                front = tuple(waylist[0])
                back = tuple(waylist[-1])
                if front != back: 
                    badrelcount += 1
                    flattened = list(arbitraryflatten(newnodelist))
                    latitudes = [element for i, element in enumerate(flattened) if i%2==0]
                    longitudes = [element for i, element in enumerate(flattened) if i%2!=0]
                    zippedlist = list(zip(latitudes, longitudes))
                    dups = [item for item, count in collections.Counter(zippedlist).items() if count > 1]
                    if dups:
                        badreldups += 1
        if bound['type'] == 'way':
            front = tuple(newnodelist[0])
            back = tuple(newnodelist[-1])
            if front != back:
                badwaycount += 1
                dups = [item for item, count in collections.Counter(newnodelist).items() if count > 1]
                if dups:
                    badwaydups += 1
                    if len(dups) > 1:
                        print(newnodelist)
                        print("")
                        print(dups)
                        print("")
                    startindex = newnodelist.index(dups[0])
                    bound['nodelist'] = bound['nodelist'][startindex:] + bound['nodelist'][:startindex + 1]            

    print("bad rel count: " + str(badrelcount))
    print("bad rel dups: " + str(badreldups))
    print("bad way count: " + str(badwaycount))
    print("bad way dups: " + str(badwaydups))

In [11]:
testnamestuff(tuple_bounding_college)

bad rel count: 185
bad rel dups: 185
bad way count: 112
bad way dups: 97


In [43]:
def testlocationdifferencebadnames(alist):
    namesdict = {}
    for bound in alist:
        nodelist = bound['nodelist']
        thename = bound['name']
        if thename not in namesdict:
            namesdict[thename] = bound
        else:
            #compare how far apart they are
            import copy
            fromnamesdict = copy.deepcopy(namesdict[thename]['nodelist'])
            frombound = copy.deepcopy(bound['nodelist'])
            templist = fromnamesdict + frombound
            flattened = list(arbitraryflatten(templist))
            latitudes = [element for i, element in enumerate(flattened) if i%2==0]
            longitudes = [element for i, element in enumerate(flattened) if i%2!=0]
            resultslat = any([abs(x - latitudes[i+1]) > 1 for i, x in enumerate(latitudes) if i != len(latitudes) - 1])
            resultslon = any([abs(x - longitudes[i+1]) > 1 for i, x in enumerate(longitudes) if i != len(longitudes) - 1])
            if resultslat or resultslon:
                print(namesdict[thename])
                print("")
                print(bound)
                return

In [33]:
testlocationdifferencebadnames(new_bounding_university)

{'name': "École Nationale d'Administration", 'type': 'relation', 'nameisid': False, 'nodelist': [[(48.5812046, 7.7367304), (48.5812141, 7.7368496), (48.5812333, 7.7370901), (48.5810919, 7.7370658), (48.581083, 7.7369563), (48.5809918, 7.7369399), (48.5809573, 7.7369337), (48.5809022, 7.7369238), (48.5809067, 7.7368738), (48.5808378, 7.7368616), (48.5808517, 7.7367176), (48.5808557, 7.7366655), (48.5812046, 7.7367304)], [(48.5803731, 7.7369873), (48.58052, 7.7371229), (48.5805554, 7.7370291), (48.5804462, 7.736925), (48.5805385, 7.7366985), (48.5807538, 7.7368954), (48.580623, 7.7372199), (48.580656, 7.7372512), (48.5806804, 7.7371943), (48.5807134, 7.7372256), (48.5808228, 7.7369644), (48.5809113, 7.7370521), (48.5809089, 7.7370605), (48.5809362, 7.7370833), (48.5809537, 7.7370979), (48.5809667, 7.7370621), (48.5813402, 7.7374945), (48.581346, 7.7375683), (48.581353, 7.7376571), (48.5812881, 7.737666), (48.5812933, 7.7377451), (48.5813607, 7.7377524), (48.5813759, 7.7379144), (48.58132